In [4]:
import numpy as np
import pandas as pd
import talib as ta


## Variables to use in this project
#### On balance volume
It is an indicator that is uses volume flow to predict sotck changes
#### Accumulation distribution line

#### Average directionnal Index

#### Moving average convergence divergence

#### Relative Strenght index

#### Stockastics oxillator

#### Standard deviation


This strategy is moving average cross over strategy, it will indicate buy or sell signal base on the cross over and
the predicted value buy the model. There is two cases we can distinguish, cross over for buy and cross over for sell. The IA will predicte de sl and the tp according to the patterns.

Above i state the technical indicator i will use in this project ( these indicator are for starting). Each indicator indication are unique. some indictor a based on the volume, other on the trend etc. First of all, we will prepare the data

In [3]:
data = pd.read_csv("../Data/eurusdheure.csv")

In [11]:
def features(data, ema_min_period, ema_max_period, adx_period, rsi_period, std_period, wma_period):
	data_ = data.copy()
	data_["close_shift"] = data_.close.shift(-1)
	data_["ema_min"] = ta.EMA(data_.close, timeperiod= ema_min_period)
	data_["ema_max"] = ta.EMA(data_.close, timeperiod= ema_max_period)
	data_["ema_max_shift_1"] = data_.ema_max.shift(1)
	data_["ema_min_shift_1"] = data_.ema_min.shift(1)
	data_["ohlc"] = (data_.open + data_.high + data_.low + 2*data_.close)/5
	data_["sell"] = np.where((data_.ema_min < data_.ema_max) & (data_.ema_max_shift_1 <  data_.ema_min_shift_1), 1, -1)
	data_["buy"] = np.where((data_.ema_min > data_.ema_max) & (data_.ema_max_shift_1 >   data_.ema_min_shift_1), 1, -1)
	data_["on_balance_volume"] = ta.OBV(data_.close, data_.tick_volume)
	data_["adx"] = ta.ADX(data_.high, data_.low, data_.close, timeperiod= adx_period)
	data_["rsi"] = ta.RSI(data_.close, timeperiod= rsi_period)
	data_["wma"] = ta.WMA(data_.ohlc, timeperiod=wma_period)
	data_["std"] = ta.STDDEV(data_.close, timeperiod=std_period)
	data_["low_shift"] = data_.low.shift(-1)
	data_["high_shift"] = data_.high.shift(-1)
	data_["close_low"] = data_.high_shift - data_.close
	data_["close_high"] = data_.close - data_.low
	data_["tp"] = np.where(data_.close_low > data_.close_high, data_.low_shift, data_.high_shift)
	data_["sl"] = np.where(data_.close_low > data_.close_high, data_.high_shift, data_.low_shift)
	data_ = data_.dropna()
	return data_

In [12]:
eurusd =  features(data, ema_max_period=75, ema_min_period=10, adx_period= 30, rsi_period=30, std_period= 30, wma_period= 30)

In [13]:
eurusd = eurusd.drop("Unnamed: 0", axis= 1)
eurusd.head()

,time,open,high,low,close,tick_volume,spread,real_volume,close_shift,ema_min,...,adx,rsi,wma,std,low_shift,high_shift,close_low,close_high,tp,sl
75,1302138000,1.43332,1.43337,1.43199,1.43238,1100,12,0,1.43152,1.432390,...,22.222109,61.470708,1.430572,0.004153,1.43115,1.43246,0.00008,0.00039,1.43246,1.43115
76,1302141600,1.43237,1.43246,1.43115,1.43152,2138,13,0,1.43141,1.432232,...,22.218362,59.412637,1.430796,0.004026,1.43072,1.43230,0.00078,0.00037,1.43072,1.43230
77,1302145200,1.43154,1.43230,1.43072,1.43141,2313,13,0,1.43041,1.432083,...,22.139870,59.150596,1.430978,0.003849,1.43011,1.43143,0.00002,0.00069,1.43143,1.43011
78,1302148800,1.43142,1.43143,1.43011,1.43041,1666,12,0,1.43104,1.431779,...,21.958979,56.794845,1.431093,0.003635,1.43020,1.43161,0.00120,0.00030,1.43020,1.43161
79,1302152400,1.43037,1.43161,1.43020,1.43104,1482,12,0,1.43039,1.431644,...,21.806215,57.887897,1.431196,0.003366,1.42970,1.43128,0.00024,0.00084,1.43128,1.42970


In [14]:
eurusd.to_csv("../Data/eurusdprepared.csv")